# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

### Not for Grading


### Learning Objectives:

At the end of the experiment, you will be able to:

* understand word2vec in action.

In this experiment we will use **Mahabharata** as our text corpus

### Setup Steps

In [ ]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "2100121" #@param {type:"string"}


In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "5142192291" #@param {type:"string"}


In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()
  
notebook= "Demo_Mahabharata" #name of the notebook
Answer = "Ungraded"
def setup():
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/week1/Saturday_Experiment/MB.txt") 
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/week1/Saturday_Experiment/word2vec.png")
    ipython.magic("sx pip3 install gensim")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "feedback_experiments_input" : Comments, "notebook" : notebook}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:        
        print(r["err"])
        return None   
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aiml.iiith.talentsprint.com/notebook_submissions")
        # print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if not Additional: 
      raise NameError
    else:
      return Additional  
  except NameError:
    print ("Please answer Additional Question")
    return None
def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError 
    else: 
      return Answer
  except NameError:
    print ("Please answer Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
    
else:
  print ("Please complete Id and Password cells before running setup")


### Import required packages

In [ ]:
# Importing nltk package
import nltk

# Downloading wordnet from NLTK to perform Stemmer
nltk.download('wordnet')

# Python library for Vector space modeling and topic modeling
import gensim

# Regular Expression
import re

# Basic Packages
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Pre-Processing and Develop Word Embeddings

* Load the Mahabharata corpus
* Stemming and removing stop words
* Generate word embeddings

In [ ]:
# Stemmer with Python nltk package
stemmer = nltk.PorterStemmer()

# Download all the stopwords from the NLTK package using nltk.download('stopwords')
nltk.download('stopwords')
from nltk.corpus import stopwords  
stopWords = set(stopwords.words('english'))

In [ ]:
MB_words = []

# Open the text file in read mode
with open("MB.txt", 'r') as file:

   # Store each line in the file as a separate element in a list
   lines = file.readlines()
   
   # Take each line from the list of lines and collect all words
   for line in lines:

        # findall() function returns a list containing all matches between a-z
        words = re.findall(r'(\b[a-z][a-z]*\b)', line.lower()) 
        
        # Stemming each word in a list, if the word is not in stopwords
        words = [stemmer.stem(word) for word in words if word not in stopWords]

        MB_words.append(words)                    
    

Get the vocabulary and vectors using gensim package. 

**min_count** ignore words that appear less than the specified count

In [ ]:
# Train the gensim model on the MB_words
model = gensim.models.Word2Vec(MB_words, min_count=120) 

In [ ]:
# Total number of words in the trained model
print("Total number of words in the trained model: ", len(model.wv.vocab.keys())) 

In [ ]:
# Number of vectors generated for each word
print("Dimensionality of word embeddings: ", len(model.wv.vectors[0]))

### Construct the word and vector list by iterating through the vocabulary of the pretrained word2vec model.

In [ ]:
words_list = [] 
vector_list = [] 

for i in model.wv.vocab:
    try :
        words_list.append(i)
        vector_list.append(model[i])
    except :
        pass
    
words_list = np.array(words_list)
vector_list = np.array(vector_list)

print(vector_list.shape, words_list.shape)

### Visualization and Plotting the reduced Word2Vec representation

* As vector_list dimensions are huge, reduce the dimensions of the vectors to 2D using PCA 

In [ ]:
# Check the shape of the vector_list before reducing its dimensions
print("Shape of the vectors_list before reducing the dimensions: ", vector_list.shape)

* Applying PCA to reduce the dimensions of the vectors

In [ ]:
# Create a 2-dimensional PCA model of the word vectors using the scikit-learn PCA class
from sklearn.decomposition import PCA

# n_components in PCA specifies the no.of dimensions
pca = PCA(n_components=2)

# Fit and transform the vectors using PCA model
reduced_vector = pca.fit_transform(vector_list)

In [ ]:
print("Shape of the vectors_list after reducing the dimensions to 2D: ", reduced_vector.shape)

* Visualize the reduced Word2Vec representation

In [ ]:
from matplotlib import pyplot as plt
colors = ['green' for i in range(len(reduced_vector))]
x = []
y = []
for vec in reduced_vector:
    x.append(vec[0])
    y.append(vec[1])
plt.figure(figsize=(28,20)) 
for i in range(len(words_list)):
    plt.scatter(x[i],y[i], color=colors[i])
    plt.annotate(words_list[i], xy=(x[i], y[i]))
plt.show()

### Choose few characters from Mahabharata and find the similar characters

* Find the location of the chosen characters in word_list


 

In [ ]:
MB_characters = ['krishna', 'arjuna', 'pandu', 'bhima', 'sakuni', 'duryodhana', 'bhishma', 'kunti', 'karna', 'madri', 'nakula', 'sahadeva', 'draupadi']

# Get the location of MB_characters from the words_list
locs = [np.where(words_list == x)[0][0] for x in MB_characters]

print("The location of the selected characters \n", locs)

* Visualization of the chosen characters in the Mahabharata

In [ ]:
# Generating the vectors for chosen characters and reducing the dimensions using PCA to plot in 2-D plane

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
for character, pos in zip(MB_characters, locs):
    char_v = model[character]
    # 'char_v' contains the vector representation of each character
    # Adding one more dimension to the 'char_v', while PCA allows only 2-d array 
    # Converting it back to 1-d array to plot the transformed vector
    value = pca.transform([char_v])[0]
    ax.plot(value[0], value[1],  "r*")
    plt.annotate(words_list[pos], xy=value, xytext=value+0.02)

plt.show()
fig.savefig('word2vec.png')


* Find the top-5 similar characters for the selected characters

In [ ]:
names= []
for character in MB_characters:
    near = model.most_similar(character, topn = 5)
    nearNames = [x[0] for x in near]
    names.append(nearNames)

pd.DataFrame(names,columns=['Similarity_1','Similarity_2','Similarity_3','Similarity_4','Similarity_5'],index = MB_characters)

## Please answer the questions below to complete the experiment:

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good, But Not Challenging for me" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "no" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook  { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")